In [ ]:
#@title Installation
!pip install timm foolbox

     |████████████████████████████████| 296kB 8.4MB/s 
     |████████████████████████████████| 1.7MB 14.3MB/s 
     |████████████████████████████████| 163kB 35.9MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
#@title Mount Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#@title Imports

import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import L2PGD

import timm

from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

from tqdm import tqdm

import cv2

import matplotlib.pyplot as plt

PyTorch Version:  1.8.1+cu101
Torchvision Version:  0.9.1+cu101


In [ ]:
#@title Global Variables
# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 32

input_size = 224

feature_extract = False

num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


In [ ]:
#@title Utils
import scipy
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('')
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            with tqdm(dataloaders[phase], unit='batch') as tepoch:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in tepoch:

                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        # Get model outputs and calculate loss
                        # Special case for inception because in training it has an auxiliary output. In train
                        #   mode we calculate the loss by summing the final output and the auxiliary output
                        #   but in testing we only consider the final output.
                        if is_inception and phase == 'train':
                            # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                            outputs, aux_outputs = model(inputs)
                            loss1 = criterion(outputs, labels)
                            loss2 = criterion(aux_outputs, labels)
                            loss = loss1 + 0.4*loss2
                        else:
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)

                        _, preds = torch.max(outputs, 1)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                if phase == 'val':
                    val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

def log_scale(array, epsilon=1e-12):
    """Log scale the input array.
    """
    array = np.abs(array)
    array += epsilon  # no zero in log
    array = np.log(array)
    return array

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

class dct(object):
  def __call__(self, sample):
    sample = sample.numpy()
    sample = scipy.fft.dct(sample, type=2, norm ="ortho", axis = 1)
    sample = scipy.fft.dct(sample, type=2, norm ="ortho", axis = 2)
    sample = log_scale(sample)
    return torch.from_numpy(sample)




In [ ]:
import tensorflow as tf
from PIL import ImageStat

class Stats(ImageStat.Stat):
  def __add__(self, other):
    # add self.h and other.h element-wise
    return Stats(list(np.add(self.h, other.h)))

def fft_normalization(dataloader): 
  statistics = None   
  toPIL=transforms.ToPILImage()
  with tqdm(dataloader, unit='batch') as tepoch:
    for data, _ in tepoch:
        for b in range(data.shape[0]):
            if statistics is None:
                statistics = Stats(toPIL(data[b]))
            else:
                statistics += Stats(toPIL(data[b]))
              
  print(f'mean:{statistics.mean}, std:{statistics.stddev}')
  return statistics.mean, statistics.stddev


In [ ]:
#@title Model
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [ ]:
data_dir = '/content/drive/MyDrive/598dataset/detector_pgd_5'
# data_dir = "/content/drive/MyDrive/598dataset/Detector/ResNet/pgd_2"
# data_dir_pgd = "/content/drive/MyDrive/598dataset/Detector/ResNet/PGDL2_0.5"

#@title Dataloader Init
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        dct()
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        dct()
    ]),
}


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['train', 'val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=1, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


fft_normalization(dataloaders['train'])



100%|██████████| 516/516 [1:19:50<00:00,  9.28s/batch]

mean:[127.35689843025278, 127.32537054175363, 127.32858003101809], std:[74.05935663161335, 74.04663458858886, 74.0499986449985]


([127.35689843025278, 127.32537054175363, 127.32858003101809],
 [74.05935663161335, 74.04663458858886, 74.0499986449985])

In [ ]:
#@title dataloader calib
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        dct(),
        transforms.Normalize([127.35689843025278, 127.32537054175363, 127.32858003101809], [74.05935663161335, 74.04663458858886, 74.0499986449985])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        dct(),
        transforms.Normalize([127.35689843025278, 127.32537054175363, 127.32858003101809], [74.05935663161335, 74.04663458858886, 74.0499986449985])
    ]),
}


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          transform[x])
                  for x in ['train', 'val']}
                  
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=1, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

In [ ]:
#@title resnet18 fft
model = model.to(device)

params_to_update = model.parameters()
if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)
optimizer = optim.Adam(params_to_update, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

criterion = nn.CrossEntropyLoss()
model_ft, hist = train_model(model, dataloaders, criterion, optimizer, num_epochs=num_epochs, is_inception=False)
torch.save(model.state_dict(), '/content/drive/MyDrive/598dataset/Detector/raw_norm_dct_norm_pgd5.pth')


  0%|          | 0/516 [00:00<?, ?batch/s]

	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.bn2.weight
	 layer

  0%|          | 0/52 [00:00<?, ?batch/s]

train Loss: 0.0186 Acc: 0.9950


  0%|          | 0/516 [00:00<?, ?batch/s]

...

val Loss: 0.0364 Acc: 0.9927


Epoch 4/4
----------


  0%|          | 0/52 [00:00<?, ?batch/s]

train Loss: 0.0006 Acc: 0.9998


100%|██████████| 52/52 [00:11<00:00,  4.43batch/s]


val Loss: 0.7610 Acc: 0.9697

Training complete in 17m 58s
Best val Acc: 0.992727
